<p>Attached Files: File week_09_txt_and_docx.zip (596.791 KB)</p>

<p>I provide a .zip containing .txt and .docx files</p>

<p>After extracting the .zip contents, use "glob" module to get the file names in a list</p>

<p>For each file, remove punctuation and stop words</p>

<p>Produce a single .dat file containing the name of each file in quotes, a colon, then a list of words separated by commas​. The list of words per file should be unique for that file. Do not include URLs or phone numbers. Words should be made lowercase.</p>

<p>Example output:</p>

&nbsp;&nbsp;&nbsp;"File 1.txt" : word1, word2, word3, word7<br>
&nbsp;&nbsp;&nbsp;"name of file.docx" : word8, word2, word1, word10<br>
&nbsp;&nbsp;&nbsp;"another file.doc" : word1, word12, word6<br>

<p>Submit only your .ipynb file.</p>


<p>When you are done with the assignment or have spent an hour on the homework (whichever comes first), please send me an email indicating which of these you have reached.</p>

&nbsp;&nbsp;&nbsp;To: benpayne@umbc.edu<br>
&nbsp;&nbsp;&nbsp;Subject: Data 601 Spring 2019 Week 10 remove stop words<br>
&nbsp;&nbsp;&nbsp;Examples of content: "I have spent 1 hour on the homework" or "I am done with the assignment in 20 minutes."<br>
<p>If you've not completed the homework, please specify what your status is.</p>

In [1]:
#One time process
#!pip install nltk
#!pip install python-docx

In [2]:
import zipfile
import nltk
import os
import glob
import chardet
from nltk.corpus import stopwords                 
import string
import re
import csv
import docx
from nltk.tokenize import sent_tokenize,word_tokenize                               #import the sentence and work tokenizer from nltk

In [3]:
#One time process
#nltk.download('stopwords')
#nltk.download('punkt')

In [4]:
#One time only
#file_2_unzip = zipfile.ZipFile('week_09_txt_and_docx.zip','r')                     #create a zipfile object to extract from
#file_2_unzip.extractall(newfolder)                                                 #extract all files from the zip file

In [5]:
en_stops = set(stopwords.words('english'))                                          #create a set of stopwords

In [6]:
#takes a list of sentences and converts them to a list of unique, lowercase, non-stop words

def unique_words(sent_tokenize_list: list) -> list: 
    
    word_set = set()                                                                #Create a set to hold words.  Set is a good choice because items can't be duplicated, but is it faster?
    for this_sent in sent_tokenize_list:                                            #Iterate through each sentence in the list
        word_tokens = word_tokenize(this_sent)                                      #Create a list of words from each sentence
        for w in word_tokens:                                                       #Iterate through each word in each sentence
            w = w.lower()                                                           #make the word lower case, because it is required and to ensure that duplicate words are not added to the set
            if (w not in en_stops):                                                 #If the word is a stop word, then do nothing.  If the word is not a stop word, then proceed.
                word_set.add(w)                                                     #Add non-stop words to the set.  Notice that sets contain unique elements only, so I don't have to test for uniqueness, is this better or worse than testing?
    return list(word_set)

In [7]:
#I got this function from the internet  https://stackoverflow.com/questions/25228106/how-to-extract-text-from-an-existing-docx-file-using-python-docx

def getText(filename):                                          
    doc = docx.Document(filename)                                                   #open the docx file 
    fullText = []                                                                   #create a list to hold the file
    for para in doc.paragraphs:                                                     #access the paragraphs in the document one at a time
        fullText.append(para.text)                                                  #convert the docx format to text and at it to the list
    return '\n'.join(fullText)                                                      #returns a string with the text of the files

In [8]:
result = {}

path = './files/'                                                                   #identify location of the data files
os.chdir(path)                                                                      #I change the path b/c I prefer the format I get from glob.  I change it back below
for filename in glob.glob('*'):                                                     #grab the files in the 'files/' subfolder and iterate through them.
    print(filename)                                                                 #print the filename so I can monitor the progress
    extension = filename.split('.')                                                 #grab the extension of the file.  There are two file types I am working with, .txt and .docx and they must be handled differently
    if extension[1] == 'txt' :                                                      #conditional identifies the .txt files
        with open (filename,'rb') as fraw:                                          #Open the file in byte format and assign it to a fraw.  I had to use this 'rb' or 'read bytes' format instead of 'rt' or 'read text' because the encoding with some of the files was not the same as my systems default, but to use the chardet detection I had to read the file in in bytes.
            text = fraw.read()                                                      #Read the file in as bytes
            encoding_format = chardet.detect(text)['encoding']                      #Determine what the file encoding is and assign the resulting encoding value from the returned dictionary to a variable
        fraw.close()                                                                #Close the opened file

        with open (filename,'rt', encoding=encoding_format) as file:                #Open the file in character format using the corresponding encoding and assign it to file variable 
            text = file.read()                                                      #Read the file in as text
            re.sub(r"\d\d\d-\d\d\d-\d\d\d\d","",text)                               #Use regular expressions module to replace all phone number formats with nothing (remove them from the file).  I got this regex from Ben's notebook.
            text = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?","",text)  #Use regular expressions module to replace all urls with nothing.  I got the regex from the following website https://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string

        text = text.translate(str.maketrans('','',string.punctuation))              #I needed to remove all punctuation.  I got the previous code from the following website:  https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
                                                                                    #       for above - the str.translate(str.maketrans) works as follows.  str.maketrans creates a translation table from x to y, but it also has a section for characters that don't get mapped.  that is where the punctuation argumaent goes.
        sent_tokenize_list = sent_tokenize(text)                                    #Use nltk to create a list of sentences
        
        result['\"' + filename + '\"'] = (', '.join(unique_words(sent_tokenize_list)))  #The prior code calls the unique_words function, which returns a list of unique strings and then joins each string in the list to create one long string and assigns it to a unique key in a dictionary based on a filename string              #https://stackoverflow.com/questions/36094979/convert-a-list-of-strings-into-one-string
        file.close()                                                                #closes the file
    elif extension[1] == 'docx' :
        text = getText(filename)                                                    #opens the docx file and returns the text as one string, assigning it to text
        re.sub(r"\d\d\d-\d\d\d-\d\d\d\d","",text)                                   #Use regular expressions module to replace all phone number formats with nothing (remove them from the file).  I got this regex from Ben's notebook.
        text = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?","",text)  #Use regular expressions module to replace all urls with nothing.  I got the regex from the following website https://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string
        text = text.translate(str.maketrans('','',string.punctuation))              #https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
                                                                                    #    for above - the str.translate(str.maketrans) works as follows.  str.maketrans creates a translation table from x to y, but it also has a section for characters that don't get mapped.  that is where the punctuation argumaent goes.
        sent_tokenize_list = sent_tokenize(text)                                    #Use nltk to create a list of sentences
        result['\"' + filename + '\"'] = str(', '.join(unique_words(sent_tokenize_list))) #The prior code calls the unique_words function, which returns a list of unique strings and then joins each string in the list to create one long string and assigns it to a unique key in a dictionary based on a filename string              #https://stackoverflow.com/questions/36094979/convert-a-list-of-strings-into-one-string
os.chdir('..')                                                                      #return to the current working directory

52256-0.txt
53031-0.txt
58108-0.txt
blind_text.txt
dr_yawn.txt
how_rubber_goods_are_made.txt
most_boring_ever.txt
most_boring_part2.txt
pg12814.txt
pg14895.txt
pg43994.txt
random_text.txt
smiley_the_bunny.txt
week_10_document1.docx
week_10_document2.docx


In [9]:
result_str = str(result)                                                                             #Create one string containing all of the strings in the dictionary
result_str = result_str.translate(str.maketrans('','',"''"))                                         #remove the remaining ''

with open('output.dat','w') as f_output:                                                             #save the text in a .dat file
    f_output.write(result_str)
f_output.close()